In [ ]:
import numpy as np
import pandas as pd
import os
from google.colab import files

uploaded = files.upload()

In [ ]:
import io
df2 = pd.read_csv(io.BytesIO(uploaded['english sentences 1.csv']))

KeyError: ignored

In [ ]:

import spacy
import inflect
import re


In [ ]:
df2 = df2.drop_duplicates(subset='Sentence', keep='first')
#df2['Words list'] = [x.split() for x in df2['Sentence']]

In [ ]:
df3 = df2

In [ ]:
nlp = spacy.load('en_core_web_sm')


In [ ]:
'''
# Define a function to clean the sentences using spaCy
def clean_sentence(sentence):
    # Parse the sentence using the loaded model
    doc = nlp(sentence)
    # Extract the token text if token is not punctuation, not a space, and not a number
    tokens = [token.text.lower() for token in doc if token.is_alpha and not token.is_space]
    # Return the cleaned sentence as a string
    return ' '.join(tokens)

# Apply the cleaning function to the 'Sentence' column of the DataFrame
df2['Cleaned_Sentence'] = df2['Sentence'][:30000].apply(clean_sentence)'''

"\n# Define a function to clean the sentences using spaCy\ndef clean_sentence(sentence):\n    # Parse the sentence using the loaded model\n    doc = nlp(sentence)\n    # Extract the token text if token is not punctuation, not a space, and not a number\n    tokens = [token.text.lower() for token in doc if token.is_alpha and not token.is_space]\n    # Return the cleaned sentence as a string\n    return ' '.join(tokens)\n\n# Apply the cleaning function to the 'Sentence' column of the DataFrame\ndf2['Cleaned_Sentence'] = df2['Sentence'][:30000].apply(clean_sentence)"

In [ ]:

# Initialize inflect engine
p = inflect.engine()

# A helper function to convert ordinals to words
def ordinal_to_words(match):
    number = int(match.group(0)[:-2])  # Convert the ordinal number to an integer
    return p.number_to_words(p.ordinal(number))  # Convert to ordinal word

# Define a function to preprocess the sentence
def preprocess_sentence(sentence):
    # Convert ordinal numbers to words
    sentence = re.sub(r'\b\d+st\b|\b\d+nd\b|\b\d+rd\b|\b\d+th\b', ordinal_to_words, sentence)

    # Convert cardinal numbers to words
    sentence = ' '.join([p.number_to_words(token) if token.isdigit() else token for token in sentence.split()])

    # Parse the sentence using the loaded model
    doc = nlp(sentence)

    # Initialize an empty list to store the tokens
    tokens = []

    # Iterate over each token
    for token in doc:
        # Keep tokens that are words (no punctuation, spaces, etc.)
        if token.is_alpha:
            tokens.append(token.text.lower())

    # Return the cleaned sentence as a string
    return ' '.join(tokens)

# Apply the preprocessing function to the 'Sentence' column of the DataFrame
df3['Cleaned_Sentence'] = df3['Sentence'][:5000].apply(preprocess_sentence)

# Now df2 has an additional column 'Cleaned_Sentence' with the cleaned sentences


In [ ]:
df4 = df2

In [ ]:
# Initialize inflect engine
p = inflect.engine()

# A helper function to convert ordinals to words
def ordinal_to_words(match):
    number = int(match.group(0)[:-2])  # Convert the ordinal number to an integer
    return p.number_to_words(p.ordinal(number))  # Convert to ordinal word

# A helper function to convert currency to words
def currency_to_words(match):
    amount = match.group(0)[1:]  # Extract the amount without the currency symbol
    currency_word = 'dollars' if match.group(0).startswith('$') else 'euros'  # You can extend this logic to other currencies
    return p.number_to_words(amount) + ' ' + currency_word

# Define a function to preprocess the sentence
def preprocess_sentence(sentence):
    # Convert currency to words
    sentence = re.sub(r'\$\d+', currency_to_words, sentence)

    # Convert ordinal numbers to words
    sentence = re.sub(r'\b\d+st\b|\b\d+nd\b|\b\d+rd\b|\b\d+th\b', ordinal_to_words, sentence)

    # Convert cardinal numbers to words
    sentence = ' '.join([p.number_to_words(token) if token.isdigit() else token for token in sentence.split()])

    # Parse the sentence using the loaded model
    doc = nlp(sentence)

    # Initialize an empty list to store the tokens
    tokens = []

    # Iterate over each token
    for token in doc:
        # Keep tokens that are words (no punctuation, spaces, etc.)
        if token.is_alpha:
            tokens.append(token.text.lower())

    # Return the cleaned sentence as a string
    return ' '.join(tokens)

# Apply the preprocessing function to the 'Sentence' column of the DataFrame
df4['Cleaned_Sentence'] = df4['Sentence'][:5000].apply(preprocess_sentence)


In [ ]:
df5 = df4
# Create a function to detect if a number was converted in the sentence
def number_converted(original, cleaned):
    return bool(re.search(r'\d', original)) and not bool(re.search(r'\d', cleaned))

# Apply the function to each row
df5['Number_Converted'] = df5.apply(lambda row: number_converted(row['Sentence'], row['Cleaned_Sentence']), axis=1)

# Filter the DataFrame to get the sentences where a number was converted
converted_numbers_df = df5[df5['Number_Converted']]

# You can then check the first few to verify
print(converted_numbers_df[['Sentence', 'Cleaned_Sentence']].head())


TypeError: ignored